# WebScraping de los resultados históricos de la Formula 1

El objetivo es recopilar en un dataset los datos básicos de todas las carreras puntuables de la Formula 1.
La Formula 1 es un deporte que se viene realizando formalmente desde 1050, donde las escuderías y los pilotos van mejorando continuamente. Cada año se realizan varias carreras en diferentes circuitos, habitualmente en diferentes países. 
Vamos a recopilar información de los resultados de la carrera puntuable para el campeonato, obteniendo la información básica sobre el circuito, el piloto, la escudería y el resultado.

In [14]:
import requests
from bs4 import BeautifulSoup, NavigableString, Tag
import csv
from datetime import datetime
import pandas as pd
import time
from tqdm import tqdm

In [2]:
# Raíz de la página sobre la que trabajaremos
url_root = "http://www.formula1.com"
# Página específica donde iniciaremos nuestra extracción de información
url_start = "https://www.formula1.com/en/results.html"

In [3]:
# Creamos un dataset vacio con todas las columnas que vamos a obtener
#df1 = pd.DataFrame(columns=['date'], dtype=str)
#df2 = pd.DataFrame(columns=['country','circuit'], dtype=str)
#df3 = pd.DataFrame(columns=['position','car_num'], dtype=int)
#df4 = pd.DataFrame(columns=['name','surname','alias', 'team'], dtype=str)
#df5 = pd.DataFrame(columns=['laps'], dtype=int)
#df6 = pd.DataFrame(columns=['duration'], dtype=str)
#df7 = pd.DataFrame(columns=['points'], dtype=int)
#Formula_1 = pd.concat([df1, df2, df3, df4, df5, df6, df7], axis=1)
Formula_1 = pd.DataFrame(columns=['date','country','circuit','position','car_num','name','surname','alias', 'team','laps','duration','points'])

Vamos a generar tres funciones diferentes, que son las que nos realizarán la conexión con la página web, verificarán si dicha conexión es correcta, y en caso afirmativo extraerán la información solicitada. 
La primera función extraerá la información de las url de cada año que ha habido en la Formula 1.
La segunda función extraerá la información de las url de cada Gran premio de un año concreto.
La tercera función extraerá la información de detalle de cada carrera carrera específica.

In [4]:
# Función extracción url relativos a los diferentes años de los que hay datos.

def F1_year_extract(link):
    # Realizamos la petición a la web de la Fórmula 1.
    connection = requests.get(link)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = connection.status_code

    # Si obtenemos los datos, realizamos el proceso, en caso contrario, terminamos.
    if statusCode == 200:
    
        # Creamos una lista vacía para alojar las url de cada año
        years_url_list = []
    
        # Descargamos la página raíz de los resultados
        content = requests.get(link).text
        soup = BeautifulSoup(content, "lxml")
        
        # Acotamos al código donde se especifican las url de cada año
        cod_url_years = soup.find('div', {'class': 'resultsarchive-filter-container'})
    
        # Acotamos a cada división que contiene la url que buscamos
        data_year = cod_url_years.find_all('li', {'class': 'resultsarchive-filter-item'})
    
        # Extraemos el año y la url donde buscaremos los resultados y lo incorporamos a la lista creada
        for dy in data_year:
            year = dy.find("span").getText()
            if year.isdigit():
                link = dy.find("a").get("href")
                url = "%s%s" % (url_root, link)
                years_url_list.append(url)  # Lista "path" años.
                
                
        return years_url_list
    else:
        print("Error de carga en la página inicial:",link)

In [5]:
# Función extracción url relativos a cada gran premio de cada año.

def F1_prix_extract(link):
    # Realizamos la petición a la web de la Fórmula 1.
    connection = requests.get(link)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = connection.status_code

    # Si obtenemos los datos, realizamos el proceso, en caso contrario, terminamos.
    if statusCode == 200:
        
        # Creamos una lista vacía para almacenar los link a cada evento concreto
        prix_url_list = []
        
        # Descargamos la página de cada año
        content = requests.get(link).text
        soup = BeautifulSoup(content, "lxml")
        
        
        # Seleccionamos la división de donde extreremos los links de los eventos.
        table = soup.find('table', {'class':'resultsarchive-table'})

        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        for row in rows:
            link = row.find('a', {'class':'dark bold ArchiveLink'}).get("href")
            country = row.find('a', {'href':link}).getText(strip=True)
            date = row.find('td',{'class':'dark hide-for-mobile'}).getText(strip=True)
            url = "%s%s" % (url_root, link)
            prix_url_list.append((country,date,url))
       
        return(prix_url_list)
      
    else:
        print("Error de carga en la página de un año concreto:",link)



In [6]:
# Función extracción información de resultados para cada gran premio.

def F1_data_extract(link):
        
    # Realizamos la petición a la web de la Fórmula 1.
    connection = requests.get(link)

    # Comprobamos que la petición nos devuelve un Status Code = 200
    statusCode = connection.status_code

    # Si obtenemos los datos, realizamos el proceso, en caso contrario, terminamos.
    if statusCode == 200:
        
        # Descargamos la página de cada evento
        content = requests.get(link).text
        soup = BeautifulSoup(content, "lxml")
        
        # Acotamos al código donde se especifica el nombre del circuito
        circuit = soup.find('span', {'class': 'circuit-info'}).getText()
        
        table = soup.find('table', {'class':'resultsarchive-table'})
        table_body = table.find('tbody')
        rows = table_body.find_all('tr')
        
        records = []
        
        for row in rows:
            cols = row.find_all('td')
            pos = cols[1].getText()
            no = cols[2].getText()
            name = cols[3].find('span',{'class':'hide-for-tablet'}).getText()
            surname = cols[3].find('span',{'class':'hide-for-mobile'}).getText()
            alias = cols[3].find('span',{'class':'uppercase hide-for-desktop'}).getText()
            team = cols[4].getText()
            laps = cols[5].getText()
            duration = cols[6].getText()
            points = cols[7].getText()
            records.append((circuit,pos,no,name,surname,alias,team,laps,duration,points))
        
        # Incorporamos un retraso de 1 segundo después de cara obtención de datos
        time.sleep(1)
        return(records)

    else:
        print("Error de carga en la página de un evento concreto:",link)


A partir de este punto está el programa que irá utilizando las funciones creadas para la recolección de informacion, y la icorporará en el dataset. Finalmente escribimos el dataset obtenido en un fichero csv.

In [7]:
# Obtenemos el listado de url de cada año.
F1_url_by_year = F1_year_extract(url_start)


In [15]:
# Obtenemos el listado de urls de cada gran premio según el año.
F1_event_url_list = []

# Vamos a extraer la información para cada url de cada año
for season in tqdm(F1_url_by_year):
    
    # Extraemos la lista de url de eventos por cada año
    url_event_list = F1_prix_extract(season)
    
    # Extraemos los datos de la lista para crear una única lista con la información que nos interesa.
    for event in url_event_list:
        country = event[0]
        date = event[1]
        link_e = event[2]
        reg = (country,date,link_e)
        # Esta lista contendrá todas las url de todos los eventos.
        F1_event_url_list.append(reg)

100%|██████████| 71/71 [00:54<00:00,  1.31it/s]


In [16]:
# Obtenemos los resultados de cada carrera

#for race in F1_event_url_list:
for race in tqdm(F1_event_url_list):
    
    # Identificamos la información que tenemos en cada elemento de la lista
    country = race[0]
    date = race[1]
    link = race[2]
    
    # Extraemos los datos de cada carrera
    data = F1_data_extract(link)
    
    #recorrer la lista data de los resultado de ese evento concreto
    for result in data:
        
        date_f = datetime.strptime(date, '%d %b %Y').date()
        
        # Incorporar en el dataset cada registro.
        new_reg = {'date':date_f,
                   'country':country, 
                   'circuit':result[0],
                   'position':result[1],
                   'car_num':result[2],
                   'name':result[3],
                   'surname':result[4],
                   'alias':result[5],
                   'team':result[6],
                   'laps':result[7],
                   'duration':result[8],
                   'points':result[9]
                  }
        Formula_1.loc[len(Formula_1)] = new_reg
    

100%|██████████| 1033/1033 [37:36<00:00,  2.18s/it]


In [17]:
# Escribimos el dataset obtenido en un fichero csv
Formula_1.to_csv('Formula_1_results.csv', index=False)